In [2]:
import numpy as np
from aocd.models import Puzzle
from itertools import product

from collections import defaultdict

aoc = Puzzle(year=2024, day=12)

In [ ]:
# use test data
raw_test = aoc.examples[0].input_data

# use real data
raw = aoc.input_data

print(raw_test)

In [ ]:
def parse_data(data):
    data = [list(d) for d in data.split("\n")]
    return np.array(data)


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

# Part 1


In [ ]:
neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0)]


def get_region(data, start):
    visited = {start}
    queue = [start]
    while queue:
        x, y = queue.pop(0)
        for dx, dy in neighbors:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < data.shape[0]
                and 0 <= ny < data.shape[1]
                and data[nx, ny] == data[x, y]
                and (nx, ny) not in visited
            ):
                queue.append((nx, ny))
                visited |= {(nx, ny)}
    return visited


def area(region):
    return len(region)


def perimeter_dict(region):
    perimeters = defaultdict(int)
    queue = list(region)
    while queue:
        x, y = queue.pop(0)
        for dx, dy in neighbors:
            nx, ny = x + dx, y + dy
            if (nx, ny) not in region:
                perimeters[(x, y)] += 1
    return perimeters


loc = real
regions = defaultdict(list)
elements = set(product(range(loc.shape[0]), range(loc.shape[1])))

while elements:
    x, y = elements.pop()
    region = get_region(loc, (x, y))
    elements -= region
    regions[loc[x, y].item()].append(region)

result = 0
for k, v in regions.items():
    for region in v:
        result += area(region) * sum(perimeter_dict(region).values())
result

In [ ]:
aoc.answer_a = result

# Part 2


In [ ]:
from shapely.geometry import MultiPoint


def count_corners(coords):
    coords = coords[:-1]
    dot_prod = [
        np.dot(
            np.array(coords[i]) - np.array(coords[i - 1]), np.array(coords[(i + 1) % len(coords)]) - np.array(coords[i])
        ).item()
        for i in range(0, len(coords))
    ]
    return np.count_nonzero(np.array(dot_prod) == 0)


def corners(region):
    poly = MultiPoint(list(region)).buffer(0.5, cap_style=3)
    corners = count_corners(list(poly.exterior.coords))
    for interior in poly.interiors:
        corners += count_corners(list(interior.coords))
    return corners


loc = real
regions = defaultdict(list)
elements = set(product(range(loc.shape[0]), range(loc.shape[1])))

while elements:
    x, y = elements.pop()
    region = get_region(loc, (x, y))
    elements -= region
    regions[loc[x, y].item()].append(region)

result = 0
for k, v in regions.items():
    for region in v:
        result += area(region) * corners(region)
result

In [ ]:
aoc.answer_b = result